# Python Group
## Lab Three: Extending Logistic Regression
### Wali Chaudhary, Bryce Shurts, & Alex Wright

## Preparation and Overview

### (Business) Use-case

As is outlined by Mr. Nansal's dataset description, a salient goal of this dataset was to provide an accurate/representative dataset regarding the failures of machines and other mechanical equipment, as real datasets of such a nature (and of a sufficient quality so as to be predictive) are difficult to find in the wild.

Naturally, this flows into the overall goal of having such a predictive dataset: increasing our ability to monitor potentially dangerous equipment for wear (or other failure conditions) and enact preventative maintenance or usage guidelines for them to prevent this from happening. While in some cases this is more of a financial and business continuity concern, e.g., the operation of an assembly line creating clothing, it can quickly spiral into health and safety concern, both for employees and others that are dependent on such tools doing their job accurately and safely: automated forge equipment, QA testing machines for foods and chemical products and tools used to create load-bearing equipment are just some of these possibilities.

As such, it is of great importance to a business financially, morally, and legally to ensure that they have reliable methods for detecting potential and upcoming failures of equipment, which this dataset could potentially help with should an accurate & generalizable classifier be created around it.

#### Citation & Acknowledgement

The *Machine Predictive Maintenance Classification* dataset is liscenced under the [Creative Commons 1.0 Universal Public Domain](https://creativecommons.org/publicdomain/zero/1.0/) and was provided by Shivam Nansal on [Kaggle](https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification).

### Preprocessing

Before we begin preprocessing, we must first break down the contents of our data and describe the nature and repersentation of each feature:

- UDI - unique ID - categorical; one-hot encoded integer
- Product ID - Another unique identifier - categorical; one-hot encoded integer
- Type - Specifies the quality variant of the product (low, medium, high) - categorical; one-hot encoded integer
- Air temperature [K] - Ambient temp in K at time of sampling - numerical; float
- Process temperature [K] - Synthetic estimate of tool temperature in K generated via stochastic method - numerical; float
- Rotational speed [rpm] - Estimated RPM of tool at time of sampling - numerical; integer
- Torque [Nm] - Output torque of tool in Nm at time of sampling - numerical; float
- Tool wear [min] - Amount of time the tool has been used for in minutes - numerical; integer
- Target - Binary of whether a failure occured - categorical; binary integer/boolean
- Failure Type - Mode of failure, if any: classification target - categorical; one-hot encoded integer

In [858]:
# Handle all imports for notebook

import pandas as pd
from pandas import (DataFrame, Series)
import numpy as np
from numpy import ndarray
from numpy.linalg import pinv
import os
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import image
import seaborn as sns
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_distances
from skimage.filters import gabor_kernel

# Comparison imports for Dr. Larson's code
from scipy.special import expit
from scipy.optimize import fmin_bfgs
from numpy import ma

In [475]:
df: DataFrame = pd.read_csv("predictive_maintenance.csv")

In [476]:
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


Immediately, we can determine that `UDI` and `Product ID` will not be useful to us, as we do not care about features that are redundant in function to the given row ID (i.e., features that are unique to each sample)

In [477]:
df.drop(["UDI", "Product ID"], axis=1, inplace=True)
df.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,M,298.1,308.6,1551,42.8,0,0,No Failure
1,L,298.2,308.7,1408,46.3,3,0,No Failure
2,L,298.1,308.5,1498,49.4,5,0,No Failure
3,L,298.2,308.6,1433,39.5,7,0,No Failure
4,L,298.2,308.7,1408,40.0,9,0,No Failure


We can also remove the `Target` feature, as we are looking to solve a multiclass classification problem and leaving this feature in will simply result in leakage where the classifier does not learn the expected relationship between the other features and `Failure Type`.

In [478]:
df.drop("Target", axis=1, inplace=True)
df.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type
0,M,298.1,308.6,1551,42.8,0,No Failure
1,L,298.2,308.7,1408,46.3,3,No Failure
2,L,298.1,308.5,1498,49.4,5,No Failure
3,L,298.2,308.6,1433,39.5,7,No Failure
4,L,298.2,308.7,1408,40.0,9,No Failure


Next, we can check for any invalid data, such as missing/null entries and non-sensical numbers (negative time or torque).

In [479]:
print("Missing entries: \n" + str(df.isna().sum()))

print("\nInvalid values: \nTorque: " + str((df["Torque [Nm]"].values < 0).any())
      + "\nTool wear: " + str((df["Tool wear [min]"].values < 0).any()))

Missing entries: 
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Failure Type               0
dtype: int64

Invalid values: 
Torque: False
Tool wear: False


Now that we have checked the data and ensured its validity, we can move on to our dummy encoding of our `Type` feature & label encoding our `Failure Type` target.

In [480]:
df = pd.concat([df, pd.get_dummies(df["Type"])], axis=1)
df.drop("Type", axis=1, inplace=True)
df["Failure Type"] = (df["Failure Type"].astype("category")).cat.codes
df.head()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type,H,L,M
0,298.1,308.6,1551,42.8,0,1,0,0,1
1,298.2,308.7,1408,46.3,3,1,0,1,0
2,298.1,308.5,1498,49.4,5,1,0,1,0
3,298.2,308.6,1433,39.5,7,1,0,1,0
4,298.2,308.7,1408,40.0,9,1,0,1,0


In [812]:
# Normalization / Standardization goes here! If required, otherwise delete this cell.

### Test/Training Split
Finally, we are presented with the option of splitting our data into a training & test split. In our case we believe this is the appropiate action to take, as we can train our agent on around 8,000 samples of our data & then cross-validate that it has learned the ground truth relationship between features & the target by attempting to inference the agents against the test data. If the accuracy drops significantly when testing against this test data, then we will know that our model has overfit the data (or otherwise failed to establish the relationship between the feature vectors & the classification vector).

In [481]:
df_train, df_test = train_test_split(df, test_size=0.2)

## Exceptional Work (Feature Extraction)

## Modeling

### One-Verus-All Logistic Regression

In [892]:
class binary_logistic_classifier:
    def __init__(self, learn_rate: float, iterations: int=20, constant: float=0.001, penalty: str=None) -> None:
        self.learn_rate = learn_rate
        self.iterations = iterations
        self.constant = constant
        self.penalty = penalty
    
    @staticmethod
    def _relu(theta: ndarray) -> ndarray:
        # ∀x∈θ max(0, x)
        return np.maximum(0, theta)
    
    @staticmethod
    def _add_bias(data: ndarray) -> ndarray:
        return np.hstack((np.ones((data.shape[0], 1)), data))
    
    def _penalty(self, gradient: ndarray) -> ndarray:
        if self.penalty == None:
            return gradient
        # LASSO
        elif self.penalty == "L1":
            return gradient + self.constant * np.sum(np.abs(self.w_))
        # RIDGE
        elif self.penalty == "L2":
            return gradient + self.constant * np.sum(self.w_**2)
        # Elastic Net
        elif self.penalty == "both":
            return gradient + self.constant * np.sum(self.w_**2) + self.constant * np.sum(np.abs(self.w_))
        else:
            raise Exception("'" + str(self.penalty) +"' not understood.")
    
    
    def predict_probability(self, data: ndarray, add_bias: bool=True) -> float:
        return self._relu((self._add_bias(data) if add_bias else data) @ self.w_)
        
    def predict(self, X: ndarray):
        return self.predict_probability(X) > 0.5
    
    def fit(self, data: ndarray, target: ndarray) -> None:
        data_bias = self._add_bias(data)
        num_samples = data_bias.shape[0] # Rows
        num_features = data_bias.shape[1] # Columns
        self.w_ = np.zeros((num_features, 1))
        
        for _ in range(self.iterations):
            gradient = self._get_gradient(data_bias, target)
            gradient = self._penalty(gradient)
            self.w_ -= self.learn_rate * gradient
            
class multiclass_logistic_classifier:
    def __init__(self, solver: type[binary_logistic_classifier], learn_rate: float, iterations: int=20, constant: float=0.001, penalty: str=None) -> None:
        self.solver = solver
        self.learn_rate = learn_rate
        self.iterations = iterations
        self.constant = constant
        self.penalty = penalty
        
    def predict_probabilities(self, X: ndarray) -> float:
        probabilities = []
        for classifier in self.fit_classifiers:
            probabilities.append(classifier.predict_probability(X).reshape((len(X), 1)))
        return np.hstack(probabilities)
        
    def predict(self, X: ndarray):
        print(self.w_)
        return np.argmax(self.predict_probabilities(X), axis=1)
        
    def fit(self, data: ndarray, targets: ndarray) -> None:
        num_samples = data.shape[0] # Rows
        num_features = data.shape[1] # Columns
        self.unique_target_classes = np.sort(np.unique(targets))
        num_target_classes = len(self.unique_target_classes)
        self.fit_classifiers = []
        
        for _, target in enumerate(self.unique_target_classes):
            target_binary = np.array(targets == target).astype(int)
            
            classifier = self.solver(learn_rate=self.learn_rate, iterations=self.iterations, constant=self.constant, penalty=self.penalty)
            classifier.fit(data, target_binary)
            
            self.fit_classifiers.append(classifier)
        self.w_ = np.hstack([classifier.w_ for classifier in self.fit_classifiers]).T

In [924]:
class steepest_descent(binary_logistic_classifier):
    def _get_gradient(self, X: ndarray, y: ndarray) -> ndarray:
        ydiff = y - self.predict_probability(X, add_bias=False) # get y difference
        gradient = np.dot(X.T, ydiff) # compute gradient using all the training examples
        gradient = gradient.reshape(self.w_.shape)
        
        return gradient * self.learn_rate
        
class stochastic_descent(binary_logistic_classifier):
    def _get_gradient(self, X: ndarray, y: ndarray) -> ndarray:
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_probability(X[idx],add_bias=False) # get y difference (now scalar)
        gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        
        return gradient * self.learn_rate
        
class newton_method(binary_logistic_classifier):
    
    def _calculate_hessian(self, X: ndarray, y: ndarray) -> ndarray:
            pred_weights = self.predict_probability(X, add_bias=False).ravel()
            relu = self._relu(pred_weights)
            grad = relu * (1 - relu)
            hessian = grad @ X ** 2
            return hessian
    
    def _get_gradient(self, X: ndarray, y: ndarray) -> ndarray: 
        pred_weights = self.predict_probability(X, add_bias=False).ravel()
        ydiff = y - pred_weights
        hessian = self._calculate_hessian(X, y)
        
        gradient = np.sum(X * ydiff[:, np.newaxis], axis=0)
        gradient = gradient.reshape(self.w_.shape)
        return pinv(hessian + np.outer(gradient, gradient)) @ gradient

# The steps for this are detailed in the notebook. Implement them. That is, use BFGS without the use of an external package (for example, do not use SciPy).
# I can only assume that this does not include the restriction of not using numpy...
class bfgs(binary_logistic_classifier):

    def _get_gradient(self, data: ndarray, target: ndarray) -> ndarray:
        target_delta = target - self.predict_probability(data, add_bias=False).ravel()
        gradient = np.mean(data * target_delta[:, np.newaxis], axis=0)
        gradient = gradient.reshape(self.w_.shape)
        return -self._penalty(gradient)
    
    def fit(self, data: ndarray, target: ndarray) -> None:
        data_bias = self._add_bias(data)
        num_features = data_bias.shape[1] # Columns
        self.w_ = np.zeros((num_features, 1)).flatten()
        
        # 1. Initial Approx. Hessian for k = 0 is an identity matrix
        k = 0
        H_k = np.identity(len(self.w_))
        Hi_k = H_k # Inverse of I is I
        
        while(np.amax(np.abs(self._get_gradient(data_bias, target))) > 1e-03 and k < self.iterations):
            # 2. Find update diretion p_k
            gradient = self._get_gradient(data_bias, target)
            p_k = -Hi_k @ gradient
            # 3. Update w
            new_weights = self.w_ + self.learn_rate * p_k
            # 4. Save scaled direction
            s_k = self.learn_rate * p_k
            # 5a. Approximate change in derivative
            self.w_ = new_weights
            v_k = self._get_gradient(data_bias, target) - gradient
            # 5b. Define u from above
            #u_k = v_k - (H_k @ s_k)
            # 6. Redefine approx. Hessian
            #H_k = H_k + ((v_k @ v_k.T)/(v_k.T @ s_k)) - (((H_k @ s_k) @ (s_k.T @ H_k))/(s_k.T @ H_k @ s_k))
            # 7. Approximate Inverse Hessian via Sherman Morris
            numerator_one = (s_k[np.newaxis, :] @ v_k[:, np.newaxis] + v_k[np.newaxis, :] @ Hi_k @ v_k[:, np.newaxis]) * (s_k[:, np.newaxis] @ s_k[np.newaxis, :])
            denominator_one = (s_k[np.newaxis, :] @ v_k[:, np.newaxis])**2
            numerator_two = Hi_k @ v_k[:, np.newaxis] @ s_k[np.newaxis, :] + s_k[:, np.newaxis] @ v_k[np.newaxis, :] @ Hi_k
            denominator_two = s_k[np.newaxis, :] @ v_k[:, np.newaxis]
            Hi_k = Hi_k + numerator_one/denominator_one - numerator_two/denominator_two
            # 8. Update k & start from step 2
            k = k + 1
            
        self.w_ = self.w_.reshape((num_features,1))

### Training

In [925]:
%%time
ds = load_iris()
X = df_train.drop("Failure Type", axis=1)
y = df_train["Failure Type"]
lr = multiclass_logistic_classifier(solver=steepest_descent, learn_rate=0.001, iterations=10, constant=0.1, penalty="L1")
lr.fit(X, y)
y_hat = lr.predict(X)
print("Accuracy: " + str(accuracy_score(y, y_hat)))

ValueError: cannot reshape array of size 72000 into shape (9,1)

In [926]:
%%time
ds = load_iris()
X = df_train.drop("Failure Type", axis=1)
y = df_train["Failure Type"]
lr = multiclass_logistic_classifier(solver=stochastic_descent, learn_rate=0.001, iterations=10, constant=0.1, penalty="L1")
lr.fit(X, y)
y_hat = lr.predict(X)
print("Accuracy: " + str(accuracy_score(y, y_hat)))

[[-2.27642895e-06 -3.04976429e-04 -3.13376429e-04 -1.36427643e-03
  -5.30764290e-05 -9.12764290e-05 -1.27642895e-06 -2.27642895e-06
  -1.27642895e-06]
 [-2.07364526e-05 -3.00453645e-03 -3.10683645e-03 -1.58037365e-02
  -4.02236453e-04 -1.26273645e-03 -1.07364526e-05 -1.77364526e-05
  -1.37364526e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [-1.56859143e-06 -3.00968591e-04 -3.12168591e-04 -1.20056859e-03
  -7.71685914e-05 -3.56859143e-06 -5.68591425e-07 -1.56859143e-06
  -5.68591425e-07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]]
Accuracy: 0.01125
CPU times: total: 31.2 ms
Wall time: 12.5 ms


In [927]:
%%time
ds = load_iris()
X = df_train.drop("Failure Type", axis=1)
y = df_train["Failure Type"]
lr = multiclass_logistic_classifier(solver=newton_method, learn_rate=0.001, iterations=10, constant=0.1, penalty="L1")
lr.fit(X, y)
y_hat = lr.predict(X)
print("Accuracy: " + str(accuracy_score(y, y_hat)))

[[-1.08690577e-10 -1.68696439e-08 -1.73275405e-08 -7.45298500e-08
  -2.98357223e-09 -6.20656888e-09 -5.74427138e-11 -8.95498090e-11
  -6.79392641e-11]
 [-1.02728579e-12 -1.51850842e-10 -1.56911425e-10 -7.76847390e-10
  -2.05505311e-11 -5.43845896e-11 -5.74847793e-13 -8.21287669e-13
  -6.76703511e-13]
 [-1.66125577e-10 -2.49724047e-08 -2.58157836e-08 -1.11716273e-07
  -4.87163989e-09 -1.72432125e-08 -8.45717652e-11 -1.60501176e-10
  -8.73839656e-11]
 [-8.07361622e-11 -1.13470369e-08 -1.17210044e-08 -6.65609699e-08
  -1.86308543e-09 -4.12129915e-09 -4.54206717e-11 -6.70808392e-11
  -5.43672626e-11]
 [-5.96013758e-10 -8.87108884e-08 -9.16519022e-08 -4.38870094e-07
  -1.32591790e-08 -3.77368168e-08 -3.64934101e-10 -5.11984792e-10
  -3.22919618e-10]
 [-2.21691640e-10 -3.18469577e-08 -3.28902816e-08 -1.66121660e-07
  -4.01482881e-09 -2.28592260e-08 -1.22126315e-10 -1.76434674e-10
  -1.55314757e-10]]
Accuracy: 0.01125
CPU times: total: 46.9 ms
Wall time: 58 ms


A breakdown of our hyperparameters and the thinking behind their selection:

- learn_rate: 0.001, A relatively high learning rate (especially compared to values like 1+e-0# which appear to be more typical in ML), which is used in conjuction with the iterations allowed to determine how far we need to be searching forward. Since we are only using 1 iteration, we need to tarvel across our function quickly to converge.
- iterations: 1, Used to match Dr. Larson's iterations used for the sake of time comparisons.
- constant: 0.1, Used to help emphasize the feature selection done by L1 in conjuction with RELU to hopefully sparsify our features quickly & find only the relevant terms. Still not too high at to not mistakenly wipe out relevant features.
- penalty: L1, Since we have few features, outliers are a concern, prompting the usage of L1 to feature select and avoiding L2 since the sum of squares will explode if it runs into an outlier and penalize all of the other features that migth be relevant.

In [930]:
%%time
ds = load_iris()
X = df_train.drop("Failure Type", axis=1)
y = df_train["Failure Type"]
lr = multiclass_logistic_classifier(solver=bfgs, learn_rate=0.001, iterations=10, constant=0.1, penalty="L1")
lr.fit(X, y)
y_hat = lr.predict(X)
print("Accuracy: " + str(accuracy_score(y, y_hat)))

[[ 1.94761252e-03 -2.47104329e-01 -2.74058171e-01  1.00016301e-01
  -9.64423844e-02  5.94341164e-02  2.53714878e-03  3.17629902e-03
   1.92257497e-03]
 [-9.61896856e-03 -8.19365159e+00 -8.43160694e+00  4.69584498e+00
  -1.05606840e+00  2.05569214e+00  1.34013532e-02  4.87586759e-03
   6.74253400e-03]
 [ 1.41587434e-04 -1.03833600e-01 -1.06437677e-01  4.52186142e-02
  -4.29847245e-03  3.57325297e-02  2.31900186e-04  1.35697453e-03
  -4.78405820e-04]
 [ 1.76289180e-03  3.89865965e-01  4.06400813e-01 -1.67529372e-01
   3.26342481e-01  2.37238729e-02  4.41219080e-04  1.54716736e-03
   5.97560705e-04]
 [ 2.27530910e-04 -3.25944564e-02 -3.32099510e-02  1.81464873e-02
  -1.09716224e-02 -7.12752208e-02  5.25417518e-04  3.65890094e-04
   8.60874060e-06]
 [ 6.08485826e-04  6.49718751e-02  6.72239448e-02 -2.01611871e-02
   2.56154342e-02 -5.50908272e-02  2.94986285e-04  4.77812377e-04
   6.17848353e-04]]
Accuracy: 0.96525
CPU times: total: 62.5 ms
Wall time: 59 ms


### Performance Comparison

In [806]:
# I'll start this section since I need to compare performance between BFGS implementations anyways
# Feel free to make stuff above this since the BFGS implementation is the "last" classifer we make if following the instructions

# Using the example BFGS class...

class BinaryLogisticRegression:
    def __init__(self, eta, iterations=20, C=0.001):
        self.eta = eta
        self.iters = iterations
        self.C = C
        # internally we will store the weights as self.w_ to keep with sklearn conventions
        
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    # convenience, private:
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # vectorized gradient calculation with regularization using L2 Norm
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C
        
        return gradient
    
    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction
    
    
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 
            # add bacause maximizing 

            
class BFGSBinaryLogisticRegression(BinaryLogisticRegression):
    
    @staticmethod
    def objective_function(w,X,y,C):
        g = expit(X @ w)
        # invert this because scipy minimizes, but we derived all formulas for maximzing
        return -np.sum(ma.log(g[y==1]))-np.sum(ma.log(1-g[y==0])) + C*sum(w**2) 
        #-np.sum(y*np.log(g)+(1-y)*np.log(1-g))

    @staticmethod
    def objective_gradient(w,X,y,C):
        g = expit(X @ w)
        ydiff = y-g # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0)
        gradient = gradient.reshape(w.shape)
        gradient[1:] += -2 * w[1:] * C
        return -gradient
    
    # just overwrite fit function
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = fmin_bfgs(self.objective_function, # what to optimize
                            np.zeros((num_features,1)), # starting point
                            fprime=self.objective_gradient, # gradient function
                            args=(Xb,y,self.C), # extra args for gradient and objective function
                            gtol=1e-03, # stopping criteria for gradient, |v_k|
                            maxiter=self.iters, # stopping criteria iterations
                            disp=False)
        
        self.w_ = self.w_.reshape((num_features,1))

        
class MultiClassLogisticRegression:
    def __init__(self, eta, iterations=20, 
                 C=0.0001, 
                 solver=BFGSBinaryLogisticRegression):
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.solver = solver
        self.classifiers_ = []
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.sort(np.unique(y)) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = []
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = np.array(y==yval).astype(int) # create a binary problem
            # train the binary classifier for this class
            
            hblr = self.solver(eta=self.eta,iterations=self.iters,C=self.C)
            hblr.fit(X,y_binary)

            # add the trained classifier to the list
            self.classifiers_.append(hblr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for hblr in self.classifiers_:
            probs.append(hblr.predict_proba(X).reshape((len(X),1))) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return np.argmax(self.predict_proba(X),axis=1) # take argmax along row


In [848]:
%%time
X = df_train.drop("Failure Type", axis=1)
y_not_binary = df_train["Failure Type"]

lr = MultiClassLogisticRegression(eta=1,
                                  iterations=10,
                                  C=0.01,
                                  solver=BFGSBinaryLogisticRegression
                                 )
lr.fit(X,y_not_binary)
print(lr)

yhat = lr.predict(X)
print('Accuracy of: ',accuracy_score(y_not_binary,yhat))

MultiClass Logistic Regression Object with coefficients:
[[-6.29163163e-04 -1.88702204e-01 -1.95025640e-01 -9.70160828e-01
  -2.50199122e-02 -6.79549317e-02 -6.44449736e-05 -3.73796938e-04
  -1.90921252e-04]
 [ 6.29913133e-04  1.88919241e-01  1.95251857e-01  9.70189856e-01
   2.47614334e-02  6.63639380e-02  6.59191740e-05  3.70128969e-04
   1.93864990e-04]
 [-6.29822879e-04 -1.88937962e-01 -1.95240897e-01 -9.70127719e-01
  -2.50633788e-02 -6.71334296e-02 -6.49639686e-05 -3.71884760e-04
  -1.92974150e-04]
 [-6.32636374e-04 -1.89781035e-01 -1.96114670e-01 -9.69696239e-01
  -2.52360183e-02 -6.83663662e-02 -6.49581991e-05 -3.76192701e-04
  -1.91485474e-04]
 [-6.30806902e-04 -1.89232053e-01 -1.95546333e-01 -9.69934373e-01
  -2.52604693e-02 -6.81285697e-02 -6.40143983e-05 -3.75382331e-04
  -1.91410173e-04]
 [-6.31007020e-04 -1.89290695e-01 -1.95607680e-01 -9.69946990e-01
  -2.52948275e-02 -6.75950952e-02 -6.45330004e-05 -3.75898759e-04
  -1.90575260e-04]]
Accuracy of:  0.96525
CPU times: tot

As we can see from the immediate comparison, we end up with both implementations of BFGS getting the same 96.5% accuracy on our dataset, albeit through a vastly different set of weights. However, we should also consider the speed: both CPU and wall times see a significant slowdown in the scipy implementation, with a speed several (3~4x) times slower than our custom implemntation. Of course, the absolute magnitude of these values is small enough that CPU jitter is going to significantly affect the results, but there are other considerations to keep in mind.

For instance, the scipy implementation computes the inverse hessian through the Sherman-Morrison algorithm just like ours, but where we use the expanded form to avoid computing matrix inverses, they instead utilize these temporary matrices, which may result in inefficiencies. We also do not use a line search algorithm to find our learning rate on each iteration (as it was not delineated as one of the steps in the instructions), and instead we tune an aggregate manually until good results are achieved. This avoids needing to run the line search, but also in effect means needing to run BFGS multiple times to optimize this hyperparameter, effectively making it more difficult to apply with good results and take longer overall.

As such, we would still recommend using the scipy implementation of BFGS, although perhaps an improved version using the expanded Sherman-Morrison formula could be used to increase speed performance?

## Testing

### Others go here...

### BFGS

In [855]:
%%time
ds = load_iris()
X = df_test.drop("Failure Type", axis=1)
y = df_test["Failure Type"]
# We already have a model/agent fit on the training data
#lr = multiclass_logistic_classifier(solver=bfgs, learn_rate=0.001, iterations=10, constant=0.1, penalty="L1")
#lr.fit(X, y) 
y_hat = lr.predict(X)
print("Accuracy: " + str(accuracy_score(y, y_hat)))

[[ 1.94761252e-03 -2.47104329e-01 -2.74058171e-01  1.00016301e-01
  -9.64423844e-02  5.94341164e-02  2.53714878e-03  3.17629902e-03
   1.92257497e-03]
 [-9.61896856e-03 -8.19365159e+00 -8.43160694e+00  4.69584498e+00
  -1.05606840e+00  2.05569214e+00  1.34013532e-02  4.87586759e-03
   6.74253400e-03]
 [ 1.41587434e-04 -1.03833600e-01 -1.06437677e-01  4.52186142e-02
  -4.29847245e-03  3.57325297e-02  2.31900186e-04  1.35697453e-03
  -4.78405820e-04]
 [ 1.76289180e-03  3.89865965e-01  4.06400813e-01 -1.67529372e-01
   3.26342481e-01  2.37238729e-02  4.41219080e-04  1.54716736e-03
   5.97560705e-04]
 [ 2.27530910e-04 -3.25944564e-02 -3.32099510e-02  1.81464873e-02
  -1.09716224e-02 -7.12752208e-02  5.25417518e-04  3.65890094e-04
   8.60874060e-06]
 [ 6.08485826e-04  6.49718751e-02  6.72239448e-02 -2.01611871e-02
   2.56154342e-02 -5.50908272e-02  2.94986285e-04  4.77812377e-04
   6.17848353e-04]]
Accuracy: 0.965
CPU times: total: 0 ns
Wall time: 4.5 ms


With only ~2000 samples the time remains, as expected, fairly small (especially since we are only inferencing and not training), but achieving an even higher accuracy on the testing dataset than the training dataset is a great result! It appears that we were able to successfully meet our goal of creating an accurate & robust classifier, at least as far as the BFGS algorithm implementation is concerned.

#### Potentially add on more to this section depending on how the other tests went? <b>@Wali</b>

## Deployment

Answer goes here.